# costPower

In [120]:
import requests
import pymongo
res=requests.get("https://sip.einvoice.nat.gov.tw/ods-main/ODS308E/download/691C0280-CEFB-488F-9E71-6AA4F39A41CD/8/1124193D-09F5-4711-AB9A-01848E3B88E4/8/?fileType=csv")
x=res.text
keys=x.replace('"',"").replace("\u3000","").replace("\U000fffb4","").replace("\r","").split("\n")[0].split(",")
values=[data.replace("\u3000","")
        .replace("\U000fffb4","")
        .replace("\U000fb56f","塭")
        .replace("\r","")
        .split(",") for data in x.replace('"',"").split("\n")[1:]]
datalist=[]
for valueList in values:
    data={}
    for key,value in zip(keys,valueList):
        data[key]=value
    datalist.append(data)
dataFinal=[]
for i in datalist:
    data={}
    if i.get('主行業別',"")=='餐飲業' and i.get('年度',"")=='2017' and i.get("消費熱度計算來源","")=="電子發票":
        data['add']=i['縣市']+i['鄉鎮市區']+i['村里']
        data['costPower']=i['銷售額指標']
        dataFinal.append(data)
        
client=pymongo.mongo_client.MongoClient("localhost",27017)
print(client.database_names())
db=client.rawData
print(db.collection_names())
collection=db.addressCoordinate
Coordinate={add['add']:{"lat":add['lat'],'lng':add['lng']} for add in list(collection.find({}))}
client.close()

for add in dataFinal:
    try:
        add['lat']=Coordinate[add['add']]['lat']
        add['lng']=Coordinate[add['add']]['lng']
        add["_id"]=add.pop('add')
    except:
#         print(add['add'])
        pass
costPower=[add for add in dataFinal if 'lat' in add]

client=pymongo.mongo_client.MongoClient("localhost",27017)
db=client.rawData
collection=db.CostPower
try:
    collection.drop()
    collection.insert_many(costPower,ordered=False)
except:
    print("some data has same _id")
client.close()

['admin', 'local', 'rawData']
['ipeenWebsite', 'Nhumnan', 'ipeenInfo', 'addressCoordinate', 'HRdata104', 'CostPower', 'Nhuman']
臺北市大安區羣賢里
臺北市信義區富臺里
臺北市萬華區糖里
臺中市大肚區蔗里
臺中市北屯區子里
臺中市東區富臺里
臺南市中西區赤崁里
臺南市安南區塭南里
臺南市安南區公塭里
臺南市麻豆區榖興里
臺南市麻豆區晉江里
高雄市三民區鼎金里
高雄市三民區鼎西里
高雄市三民區本館里
高雄市三民區灣子里
高雄市三民區港東里
高雄市三民區港西里
高雄市三民區建東里
高雄市三民區德東里
高雄市三民區德西里
高雄市三民區千歲里
高雄市三民區興德里
高雄市三民區正興里
高雄市三民區立德里
高雄市三民區灣華里
高雄市三民區灣中里
高雄市三民區灣復里
高雄市三民區灣興里
高雄市三民區寶珠里
高雄市三民區民享里
高雄市三民區精華里
高雄市三民區寶龍里
高雄市三民區安邦里
高雄市三民區灣利里
高雄市三民區達明里
高雄市三民區灣愛里
高雄市三民區安和里
高雄市三民區德智里
高雄市三民區寶興里
高雄市三民區安寧里
高雄市三民區鼎盛里
高雄市三民區鼎泰里
高雄市三民區本文里
高雄市三民區寶盛里
高雄市三民區寶中里
高雄市三民區寶玉里
高雄市三民區灣成里
高雄市三民區寶國里
高雄市三民區安發里
高雄市三民區鼎中里
高雄市三民區寶業里
高雄市三民區寶慶里
高雄市三民區寶泰里
高雄市三民區寶安里
高雄市三民區本安里
高雄市三民區本上里
高雄市三民區本揚里
高雄市三民區鼎力里
高雄市三民區本和里
高雄市內門區內豐里
高雄市岡山區臺上里
高雄市岡山區爲隨里
高雄市鳳山區成功里
高雄市鳳山區和興里
高雄市鳳山區南興里
高雄市鳳山區鎮南里
高雄市鳳山區興仁里
高雄市鳳山區鎮北里
高雄市鳳山區縣衙里
高雄市鳳山區文山里
高雄市鳳山區文英里
高雄市鳳山區中和里
高雄市鳳山區忠義里
高雄市鳳山區縣口里
高雄市鳳山區光明里
高雄市鳳山區三民里
高雄市鳳山區鎮東里
高雄市鳳山區新興里
高雄市鳳山區過埤里
高雄市鳳山區生明里
高雄市鳳山區瑞竹里
高雄市鳳山區老爺里
高雄市鳳山區五福里
高雄市鳳山區忠誠里
高雄市鳳山區北門里
高雄市鳳山區富甲里
高雄市鳳山區一甲里
高雄市鳳山區

# humanDensity

In [118]:
import requests
import json
import pymongo
res=requests.get("https://quality.data.gov.tw/dq_download_json.php?nid=32973&md5_url=52629fae1f1fed4ef81fcf706878dd80")
x=json.loads(res.text)
data=[{"add":add['site_id'].replace("　","").replace("\U000fffb5","五")+add['village'].replace(" ","").replace("\U000fffb5","五"),'Nhuman':add['people_total']}for add in x[1:]]
client=pymongo.mongo_client.MongoClient("localhost",27017)
print(client.database_names())
db=client.rawData
print(db.collection_names())
collection=db.addressCoordinate
Coordinate={add['add']:{"lat":add['lat'],'lng':add['lng']} for add in list(collection.find({}))}
client.close()

for add in data:
    try:
        add['lat']=Coordinate[add['add']]['lat']
        add['lng']=Coordinate[add['add']]['lng']
        add["_id"]=add.pop('add')
    except:
#         print(add['add'])
        pass
Nhuman=[add for add in data if 'lat' in add]
client=pymongo.mongo_client.MongoClient("localhost",27017)
db=client.rawData
collection=db.Nhuman
try:
    collection.drop()
    collection.insert_many(Nhuman,ordered=False)
except:
    print("some data has same _id")
client.close()

['admin', 'local', 'rawData']
['ipeenWebsite', 'Nhumnan', 'ipeenInfo', 'addressCoordinate', 'HRdata104', 'CostPower', 'Nhuman']
新北市板橋區公?里
新北市中和區瓦??里
新北市中和區灰??里
新北市永和區新?里
新北市新店區五?里
新北市樹林區??寮里
新北市三峽區永?里
新北市瑞芳區爪?里
新北市瑞芳區?新里
新北市瑞芳區?洞里
新北市土城區?廷里
新北市坪林區石??里
新北市萬里區崁?里
臺北市萬華區糖?里
桃園市大園區?林里
桃園市新屋區?榔里
臺中市西區公?里
臺中市西區?龍里
臺中市北屯區?子里
臺中市清水區?榔里
臺中市外埔區?子里
臺中市大安區龜??里
臺中市大肚區蔗?里
臺南市新營區舊?里
臺南市後壁區後?里
臺南市麻豆區?江里
臺南市麻豆區寮?里
臺南市官田區南?里
臺南市佳里區頂?里
臺南市西港區??林里
臺南市七股區?埕里
臺南市七股區?榔里
臺南市新化區山?里
臺南市新化區??拔里
臺南市山上區玉?里
臺南市左鎮區榮和里
臺南市龍崎區石??里
臺南市永康區?行里
臺南市永康區?洲里
臺南市安南區??南里
臺南市安南區?田里
臺南市安南區公??里
高雄市左營區?北里
高雄市左營區?南里
高雄市三民一德智里
高雄市三民一德東里
高雄市三民一德行里
高雄市三民一德西里
高雄市三民一千歲里
高雄市鳳山一生明里
高雄市鳳山一誠德里
高雄市鳳山一文華里
高雄市鳳山一武松里
高雄市鳳山一誠智里
高雄市鳳山二福誠里
高雄市鳥松區?埔里
高雄市阿蓮區?山里
高雄市湖內區公?里
高雄市旗山區永和里
新竹縣竹北市中崙里
新竹縣竹東鎮上?里
新竹縣竹東鎮?林里
新竹縣北埔鄉水?村
苗栗縣頭份市流東里
苗栗縣頭份市珊湖里
苗栗縣頭份市斗煥里
苗栗縣頭份市新華里
苗栗縣頭份市興隆里
苗栗縣頭份市上埔里
苗栗縣頭份市土牛里
苗栗縣頭份市頭份里
苗栗縣頭份市忠孝里
苗栗縣頭份市仁愛里
苗栗縣頭份市信義里
苗栗縣頭份市和平里
苗栗縣頭份市民族里
苗栗縣頭份市東庄里
苗栗縣頭份市後庄里
苗栗縣頭份市山下里
苗栗縣頭份市蟠桃里
苗栗縣頭份市田寮里
苗栗縣頭份市蘆竹里
苗栗縣頭份市上興里
苗栗縣頭份市下興里
苗栗縣頭份市尖山里
苗栗縣頭份市尖下里
苗栗縣頭

In [119]:
client=pymongo.mongo_client.MongoClient("localhost",27017)
db=client.rawData
collection=db.Nhuman
len(list(collection.find({})))

7659

In [134]:
client=pymongo.mongo_client.MongoClient("172.20.26.39",27017)
print(client.database_names())
db=client.rawData
print(db.collection_names())
collection=db.addressCoordinate
Coordinate=list(collection.find({}))
for i in Coordinate:
    del i["_id"]
import json
with open('../data/Coordinate.json','w') as f:
    json.dump(Coordinate,f)

['admin', 'local', 'rawData']
['ipeenWebsite', 'Nhumnan', 'ipeenInfo', 'addressCoordinate', 'HRdata104', 'CostPower', 'Nhuman']


In [1]:
import json
import pymongo
with open('../data/Coordinate.json') as f:
    coordinate=json.load(f)
    
client=pymongo.mongo_client.MongoClient("localhost",27017)
db=client.rawData
collection=db.addressCoordinate
# collection.insert_many(coordinate)

In [6]:
list(collection.find({'lat':{"$gt":25.1}}))

[{'_id': ObjectId('5a65ac6080b1101b209ceeb9'),
  'add': '新北市淡水區中和里',
  'lat': 25.2272569,
  'lng': 121.4884624},
 {'_id': ObjectId('5a65ac6080b1101b209ceeba'),
  'add': '新北市淡水區屯山里',
  'lat': 25.2406523,
  'lng': 121.4541881},
 {'_id': ObjectId('5a65ac6080b1101b209ceebb'),
  'add': '新北市淡水區賢孝里',
  'lat': 25.2286934,
  'lng': 121.4599012},
 {'_id': ObjectId('5a65ac6080b1101b209ceebc'),
  'add': '新北市淡水區興仁里',
  'lat': 25.2194296,
  'lng': 121.4541881},
 {'_id': ObjectId('5a65ac6080b1101b209ceebd'),
  'add': '新北市淡水區蕃薯里',
  'lat': 25.2140395,
  'lng': 121.4770388},
 {'_id': ObjectId('5a65ac6080b1101b209ceebe'),
  'add': '新北市淡水區義山里',
  'lat': 25.2062045,
  'lng': 121.442761},
 {'_id': ObjectId('5a65ac6080b1101b209ceebf'),
  'add': '新北市淡水區忠山里',
  'lat': 25.2037873,
  'lng': 121.4698984},
 {'_id': ObjectId('5a65ac6080b1101b209ceec0'),
  'add': '新北市淡水區崁頂里',
  'lat': 25.1982827,
  'lng': 121.4313328},
 {'_id': ObjectId('5a65ac6080b1101b209ceec1'),
  'add': '新北市淡水區埤島里',
  'lat': 25.1928997,
  'lng'

In [ ]:
# pymongo.mongo_client.MongoClient("localhost",27017,username='j122085',password='850605')
# >>> from pymongo import MongoClient
# >>> client = MongoClient('example.com',
# ...                      username='user',
# ...                      password='password',
# ...                      authSource='the_database',
# ...                      authMechanism='SCRAM-SHA-1')